In [20]:
import pandas as pd
import requests

In [22]:
visits_data = pd.read_csv('visits_1k.csv')
regs_data = pd.read_csv('regs_1k.csv')

visits_data.describe()

,uuid,platform,user_agent,date
count,1000,1000,1000,1000
unique,519,3,28,996
top,251a0926-ece3-4d77-aa42-ab569fdf9fe2,web,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2023-03-01T08:01:45
freq,4,954,71,2


In [24]:
regs_data.describe()

,user_id
count,1.000000e+03
mean,4.488623e+06
std,2.620568e+06
min,2.236800e+04
25%,2.235489e+06
50%,4.473044e+06
75%,6.779707e+06
max,8.881772e+06


In [26]:
API_URL = "https://data-charts-api.hexlet.app"
DATE_BEGIN = '2023-03-01'
DATE_END = '2023-09-01'

response = requests.get(f"{API_URL}/visits?begin={DATE_BEGIN}&end={DATE_END}")

data = response.json()
df_visits = pd.DataFrame(data)
df_visits.head()

,datetime,platform,user_agent,visit_id
0,"Wed, 01 Mar 2023 10:36:22 GMT",web,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,1de9ea66-70d3-4a1f-8735-df5ef7697fb9
1,"Wed, 01 Mar 2023 06:25:00 GMT",web,Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7...,f149f542-e935-4870-9734-6b4501eaf614
2,"Wed, 01 Mar 2023 10:26:13 GMT",web,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,08f0ebd4-950c-4dd9-8e97-b5bdf073eed1
3,"Wed, 01 Mar 2023 12:33:06 GMT",web,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,19322fed-157c-49c6-b16e-2d5cabeb9592
4,"Wed, 01 Mar 2023 01:38:35 GMT",web,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,04762a22-3c9f-40c9-9ac9-6628c4381836


In [27]:
response = requests.get(f"{API_URL}/registrations?begin={DATE_BEGIN}&end={DATE_END}")

data = response.json()
df_registrations = pd.DataFrame(data)
df_registrations.head()

,datetime,email,platform,registration_type,user_id
0,"Wed, 01 Mar 2023 07:40:13 GMT",ebyrd@example.org,web,google,2e0f6bb8-b029-4f45-a786-2b53990d37f1
1,"Wed, 01 Mar 2023 13:14:00 GMT",knightgerald@example.org,web,email,f007f97c-9d8b-48b5-af08-119bb8f6d9b6
2,"Wed, 01 Mar 2023 03:05:50 GMT",cherylthompson@example.com,web,apple,24ff46ae-32b3-4a74-8f27-7cf0b8f32f15
3,"Wed, 01 Mar 2023 00:04:47 GMT",halldavid@example.org,web,email,3e9914e1-5d73-4c23-b25d-b59a3aeb2b60
4,"Wed, 01 Mar 2023 18:31:52 GMT",denise86@example.net,web,google,27f875fc-f8ce-4aeb-8722-0ecb283d0760


#Шаг 3

##Расчет метрик

###Сгруппируйте данные визитов по датам и платформам

In [46]:
response = requests.get(f"{API_URL}/visits?begin={DATE_BEGIN}&end={DATE_END}")
data = response.json()

df_visits = pd.DataFrame(data)

df_visits['datetime'] = pd.to_datetime(df_visits['datetime'])

idx = df_visits.groupby('visit_id')['datetime'].idxmax()

df_filtered_visits = df_visits.loc[idx]

df_filtered_visits = df_filtered_visits[~df_filtered_visits['user_agent'].str.contains('bot', case=False)]

df_filtered_visits['datetime'] = df_filtered_visits['datetime'].dt.strftime('%Y-%m-%d')

df_grouped = df_filtered_visits.groupby(['datetime', 'platform']).size().reset_index(name='visits')

df_grouped.columns = ['date_group', 'platform', 'visits']

df_visit_grouped = df_grouped.sort_values(by='date_group')

df_visit_grouped.head(1000000)

,date_group,platform,visits
0,2023-03-01,android,75
1,2023-03-01,ios,22
2,2023-03-01,web,279
3,2023-03-02,android,67
4,2023-03-02,ios,31
...,...,...,...
546,2023-08-30,android,35
548,2023-08-30,web,1227
550,2023-08-31,ios,50
549,2023-08-31,android,57


In [30]:
df_visit_grouped.describe(include='all')

,date_group,platform,visits
count,552,552,552.000000
unique,184,3,NaN
top,2023-03-01,android,NaN
freq,3,184,NaN
mean,NaN,NaN,251.273551
std,NaN,NaN,309.435871
min,NaN,NaN,7.000000
25%,NaN,NaN,36.000000
50%,NaN,NaN,74.500000
75%,NaN,NaN,473.250000


###Сгруппируйте также данные регистраций по датам и платформам

In [32]:
response = requests.get(f"{API_URL}/registrations?begin={DATE_BEGIN}&end={DATE_END}")
data = response.json()

df_registrations = pd.DataFrame(data)

df_registrations['date_group'] = pd.to_datetime(df_registrations['datetime']).dt.strftime('%Y-%m-%d')

df_grouped = df_registrations.groupby(['date_group', 'platform']).size().reset_index(name='registrations')

df_registrations_grouped = df_grouped.sort_values(by=['date_group'])

df_registrations_grouped.head(1000000)

,date_group,platform,registrations
0,2023-03-01,android,61
1,2023-03-01,ios,18
2,2023-03-01,web,8
3,2023-03-02,android,59
4,2023-03-02,ios,24
...,...,...,...
546,2023-08-30,android,27
548,2023-08-30,web,34
550,2023-08-31,ios,36
549,2023-08-31,android,42


In [33]:
df_registrations_grouped.describe(include='all')

,date_group,platform,registrations
count,552,552,552.000000
unique,184,3,NaN
top,2023-03-01,android,NaN
freq,3,184,NaN
mean,NaN,NaN,39.557971
std,NaN,NaN,22.078237
min,NaN,NaN,6.000000
25%,NaN,NaN,23.000000
50%,NaN,NaN,34.000000
75%,NaN,NaN,54.000000


###Объедините датайфреймы, сделайте итоговый датафрейм с расчетом конверсии
###Сохраните датафрейм в формате JSON conversion.json

In [35]:
result_df = pd.merge(df_visit_grouped, df_registrations_grouped, on=['date_group', 'platform'], how='outer')

result_df['conversion'] = (result_df['registrations'] / result_df['visits']) * 100

result_df['conversion'] = result_df['conversion'].round(2)

df_conversion = result_df.sort_values(by='date_group')

df_conversion.to_json('conversion.json')

df_conversion.head(1000000)

,date_group,platform,visits,registrations,conversion
0,2023-03-01,android,75,61,81.33
1,2023-03-01,ios,22,18,81.82
2,2023-03-01,web,279,8,2.87
3,2023-03-02,android,67,59,88.06
4,2023-03-02,ios,31,24,77.42
...,...,...,...,...,...
546,2023-08-30,ios,66,40,60.61
548,2023-08-30,web,1227,34,2.77
550,2023-08-31,android,57,42,73.68
549,2023-08-31,ios,50,36,72.00


In [36]:
df_conversion.describe(include='all')

,date_group,platform,visits,registrations,conversion
count,552,552,552.000000,552.000000,552.000000
unique,184,3,NaN,NaN,NaN
top,2023-03-01,android,NaN,NaN,NaN
freq,3,184,NaN,NaN,NaN
mean,NaN,NaN,251.273551,39.557971,52.835851
std,NaN,NaN,309.435871,22.078237,34.093892
min,NaN,NaN,7.000000,6.000000,2.250000
25%,NaN,NaN,36.000000,23.000000,7.387500
50%,NaN,NaN,74.500000,34.000000,70.480000
75%,NaN,NaN,473.250000,54.000000,80.680000


#Шаг 4

##Добавляем рекламы

###В этом шаге добавим данные по рекламным кампаниям

In [38]:
df_ads = pd.read_csv('ads.csv')

df_ads['date'] = pd.to_datetime(df_ads['date'])

df_ads['date'] = df_ads['date'].dt.strftime('%Y-%m-%d')

df_ads.rename(columns={'date': 'date_group'}, inplace=True)

df_ads.head(1000000)


,date_group,utm_source,utm_medium,utm_campaign,cost
0,2023-03-01,google,cpc,advanced_algorithms_series,212
1,2023-03-02,google,cpc,advanced_algorithms_series,252
2,2023-03-03,google,cpc,advanced_algorithms_series,202
3,2023-03-04,google,cpc,advanced_algorithms_series,223
4,2023-03-05,google,cpc,advanced_algorithms_series,265
...,...,...,...,...,...
154,2023-09-13,vk,social,intro_to_python_course,277
155,2023-09-14,vk,social,intro_to_python_course,221
156,2023-09-15,vk,social,intro_to_python_course,175
157,2023-09-16,vk,social,intro_to_python_course,224


In [40]:
df_ads.describe(include='all')

,date_group,utm_source,utm_medium,utm_campaign,cost
count,159,159,159,159,159.000000
unique,159,3,2,5,NaN
top,2023-03-01,google,cpc,virtual_reality_workshop,NaN
freq,1,73,116,43,NaN
mean,NaN,NaN,NaN,NaN,194.836478
std,NaN,NaN,NaN,NaN,60.272154
min,NaN,NaN,NaN,NaN,100.000000
25%,NaN,NaN,NaN,NaN,143.000000
50%,NaN,NaN,NaN,NaN,190.000000
75%,NaN,NaN,NaN,NaN,252.000000


In [41]:
df_ads.rename(columns={'date': 'date_group', 'utm_campaign': 'campaign'}, inplace=True)

df_ads_group = df_ads.groupby(['date_group', 'campaign']).agg({
    'cost': 'sum'
}).reset_index()[['date_group', 'cost', 'campaign']]

df_ads_group.head(1000000)

,date_group,cost,campaign
0,2023-03-01,212,advanced_algorithms_series
1,2023-03-02,252,advanced_algorithms_series
2,2023-03-03,202,advanced_algorithms_series
3,2023-03-04,223,advanced_algorithms_series
4,2023-03-05,265,advanced_algorithms_series
...,...,...,...
154,2023-09-13,277,intro_to_python_course
155,2023-09-14,221,intro_to_python_course
156,2023-09-15,175,intro_to_python_course
157,2023-09-16,224,intro_to_python_course


In [42]:
df_ads_group.describe(include='all')

,date_group,cost,campaign
count,159,159.000000,159
unique,159,NaN,5
top,2023-03-01,NaN,virtual_reality_workshop
freq,1,NaN,43
mean,NaN,194.836478,NaN
std,NaN,60.272154,NaN
min,NaN,100.000000,NaN
25%,NaN,143.000000,NaN
50%,NaN,190.000000,NaN
75%,NaN,252.000000,NaN


In [57]:
df_out = pd.merge(df_ads_group, df_conversion, on='date_group', how='outer', suffixes=('_ads', '_conversion'))
df_out['cost'].fillna(0, inplace=True)

df_out['utm_campaign'] = df_out['campaign']
column_order = ['date_group', 'visits', 'registrations', 'conversion', 'cost', 'utm_campaign']
df_out = df_out[column_order]

df_out.loc[df_out['utm_campaign'].isnull(), 'utm_campaign'] = 'none'

df_out.sort_values(by='date_group', inplace=True)
df_out.reset_index(drop=True, inplace=True)
DATE_BEGIN = pd.to_datetime("2023-03-01")
END = pd.to_datetime("2023-09-01") - pd.Timedelta(days=1)
df_out['date_group'] = pd.to_datetime(df_out['date_group'])
df_out = df_out[(df_out['date_group'] >= DATE_BEGIN) & (df_out['date_group'] <= DATE_END)]

df_out.head(1000000)

,date_group,visits,registrations,conversion,cost,utm_campaign
0,2023-03-01,75.0,61.0,81.33,212.0,advanced_algorithms_series
1,2023-03-01,22.0,18.0,81.82,212.0,advanced_algorithms_series
2,2023-03-01,279.0,8.0,2.87,212.0,advanced_algorithms_series
3,2023-03-02,67.0,59.0,88.06,252.0,advanced_algorithms_series
4,2023-03-02,31.0,24.0,77.42,252.0,advanced_algorithms_series
...,...,...,...,...,...,...
548,2023-08-30,1227.0,34.0,2.77,109.0,intro_to_python_course
549,2023-08-31,721.0,46.0,6.38,215.0,intro_to_python_course
550,2023-08-31,57.0,42.0,73.68,215.0,intro_to_python_course
551,2023-08-31,50.0,36.0,72.00,215.0,intro_to_python_course


In [60]:
df_out.describe(include='all')

,date_group,visits,registrations,conversion,cost,utm_campaign
count,553,552.000000,552.000000,552.000000,553.000000,553
unique,NaN,NaN,NaN,NaN,NaN,6
top,NaN,NaN,NaN,NaN,NaN,virtual_reality_workshop
freq,NaN,NaN,NaN,NaN,NaN,129
mean,2023-05-31 16:00:52.079566080,251.273551,39.557971,52.835851,149.768535,NaN
min,2023-03-01 00:00:00,7.000000,6.000000,2.250000,0.000000,NaN
25%,2023-04-16 00:00:00,36.000000,23.000000,7.387500,103.000000,NaN
50%,2023-06-01 00:00:00,74.500000,34.000000,70.480000,164.000000,NaN
75%,2023-07-17 00:00:00,473.250000,54.000000,80.680000,233.000000,NaN
max,2023-09-01 00:00:00,1295.000000,114.000000,96.150000,298.000000,NaN


In [62]:
df_out.to_json("ads.json")